In [77]:
import re
import timeit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier)

%matplotlib inline

In [2]:
df = pd.DataFrame.from_csv('train.csv')

In [28]:
len(df)

12096

In [3]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [4]:
df.describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
count,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,...,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000,12096.000000
mean,2757.419478,155.577794,16.372189,227.898810,50.540427,1765.400050,212.920304,218.910714,134.888476,1557.455771,...,0.044643,0.041005,0.001240,0.006200,0.000661,0.001984,0.046131,0.043237,0.027943,3.936508
std,411.173336,110.091757,8.488352,207.914221,60.701726,1382.223276,30.264925,22.889814,45.753225,1142.306119,...,0.206527,0.198311,0.035194,0.078501,0.025710,0.044501,0.209777,0.203400,0.164817,2.008400
min,1863.000000,0.000000,0.000000,0.000000,-146.000000,0.000000,0.000000,99.000000,0.000000,30.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2393.000000,63.000000,10.000000,67.000000,5.000000,764.000000,197.000000,207.000000,107.000000,750.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,2766.000000,124.000000,15.000000,180.000000,32.000000,1332.000000,220.000000,223.000000,138.000000,1296.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
75%,3100.000000,259.000000,22.000000,330.000000,79.000000,2389.500000,235.000000,235.000000,167.000000,2052.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
max,3849.000000,360.000000,52.000000,1343.000000,554.000000,6890.000000,254.000000,254.000000,248.000000,6993.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


split X/Y and Drop redundant columns

In [11]:
REDUNDANT_COLS = ['Soil_Type7', 'Soil_Type15']
Y_COLUMN_NAME = Y_COLUMN_NAME = 'Cover_Type'
X = df.drop(REDUNDANT_COLS+[Y_COLUMN_NAME], axis=1)
Y = df[[Y_COLUMN_NAME]]

In [34]:
def evaluate(pipe, params, x, y):
    grid = GridSearchCV(pipe, params, error_score=0.0)
    grid.fit(x, y)
    print('Best score %.3f %%' % grid.best_score_)
    print('Best params', grid.best_params_)
    print('Mean fit time', 
        round(grid.cv_results_['mean_fit_time'].mean(), 3))
    print('Mean score time', 
        round(grid.cv_results_['mean_score_time'].mean(), 3))

Note there are too many classes for SVM  
Also the percentages are wrong; should multiply by 100  

### KNN

In [50]:
pipeline = Pipeline([('scale', StandardScaler()), 
                     ('classifier', KNeighborsClassifier())])
params = {'classifier__n_neighbors': [1, 3, 5, 7]}
evaluate(pipeline, params, X, Y['Cover_Type'])

Best score 0.565 %
Best params {'classifier__n_neighbors': 7}
Mean fit time 0.113
Mean score time 0.932


### RandomForest

In [51]:
pipeline = Pipeline([('scale', StandardScaler()), 
                     ('classifier', RandomForestClassifier())])
params = {'classifier__n_estimators': [10, 50, 100], 'classifier__max_depth': [None, 1, 3, 5, 7]}
evaluate(pipeline, params, X, Y['Cover_Type'])

Best score 0.627 %
Best params {'classifier__max_depth': None, 'classifier__n_estimators': 100}
Mean fit time 0.217
Mean score time 0.019


### DecisionTree

In [52]:
pipeline = Pipeline([('scale', StandardScaler()), 
                     ('classifier', DecisionTreeClassifier())])
params = {'classifier__max_depth': [None, 1, 3, 5, 7]}
evaluate(pipeline, params, X, Y['Cover_Type'])

Best score 0.566 %
Best params {'classifier__max_depth': 5}
Mean fit time 0.034
Mean score time 0.002


### LogisticRegression

In [55]:
pipeline = Pipeline([('scale', StandardScaler()), 
                     ('classifier', LogisticRegression())])
params = {'classifier__C': [1e-1, 1e0, 1e1, 1e2], 'classifier__penalty': ['l1', 'l2']}
evaluate(pipeline, params, X, Y['Cover_Type'])

Best score 0.530 %
Best params {'classifier__C': 0.1, 'classifier__penalty': 'l1'}
Mean fit time 24.149
Mean score time 0.02


### AdaBoostClassifier

In [59]:
pipeline = Pipeline([('scale', StandardScaler()), 
                     ('classifier', AdaBoostClassifier())])
params = {'classifier__n_estimators': [100, 200, 300, 400, 500], 
          'classifier__learning_rate': [.25, .5, .75, 9]}
evaluate(pipeline, params, X, Y['Cover_Type'])

Best score 0.370 %
Best params {'classifier__learning_rate': 0.25, 'classifier__n_estimators': 100}
Mean fit time 2.279
Mean score time 0.306


### GradientBoostingClassifier

In [61]:
pipeline = Pipeline([('scale', StandardScaler()), 
                     ('classifier', GradientBoostingClassifier(min_samples_leaf=2, verbose=0))])
params = {'classifier__n_estimators': [300, 400, 500, 600, 700], 
          'classifier__max_depth': [1, 3, 5, 7]}
evaluate(pipeline, params, X, Y['Cover_Type'])

KeyboardInterrupt: 

`Decision Tree` fits the fastest  
`Random Forest` has the best performance  
`Logistic Regression`/`Gradient Boosting` takes forever to train  

### Ensemble

Ideally, the models included in the ensemble should have high negative correlation  
How do I measure correlation between models?  

Try building an ensemble with one classifier and no cross validation

In [71]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X, Y[Y_COLUMN_NAME])

In [107]:
ensemble_X = np.concatenate([knn.predict(X)])

In [108]:
ensemble_X

array([5, 5, 2, ..., 6, 5, 5])

In [109]:
ensemble_X = np.expand_dims(ensemble_X, axis=0)
ensemble_X

array([[5, 5, 2, ..., 6, 5, 5]])

In [110]:
ensemble_X.shape

(1, 12096)

In [111]:
ensemble_X.shape, Y[Y_COLUMN_NAME].shape

((1, 12096), (12096,))

In [113]:
classifier = XGBClassifier(
    n_estimators=2000,
    max_depth=4,
    min_child_weight=2,
    gamma=0.9,
    subsample=0.8,
    colsample_bytree=.8,
    nthread=-1,
    # objective='multi:softprob',
    scale_pos_weight=1)
classifier.fit(ensemble_X, Y[Y_COLUMN_NAME])

XGBoostError: b'[03:16:36] src/objective/multiclass_obj.cc:43: Check failed: preds->Size() == (static_cast<size_t>(param_.num_class) * info.labels_.size()) SoftmaxMultiClassObj: label size and pred size does not match\n\nStack trace returned 10 entries:\n[bt] (0) /home/k8tems/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(dmlc::StackTrace[abi:cxx11]()+0x56) [0x7f3b6d3621b6]\n[bt] (1) /home/k8tems/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(xgboost::obj::SoftmaxMultiClassObj::GetGradient(xgboost::HostDeviceVector<float>*, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*)+0x34f) [0x7f3b6d3fcdbf]\n[bt] (2) /home/k8tems/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*)+0x311) [0x7f3b6d36e9a1]\n[bt] (3) /home/k8tems/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(XGBoosterUpdateOneIter+0x27) [0x7f3b6d51

num_predictions of one classifier == num_classes * num_labels ??  
Unsure; TODO: figure out tomorrow  